# Query sample MySQL database

In [1]:
import os.path
import sqlite3
import zipfile
from pathlib import Path

import requests

## Créer une base de données SQLite

In [2]:
import os

In [3]:
os.getcwd()

'/Users/charalambos/OneDrive/Documents/ULB/Bloc2/Traitement automatique de corpus - STIC-B545/RepoGHCK/tac/module1'

In [4]:
data_path = '/Volumes/Macintosh HD/Data/ADB'

In [5]:
if not os.path.isfile(f'{data_path}/db/chinook.db'): 

    # Définition de variables
    url = "https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip"
    filename = url.split("/")[-1]
    Path("data/db").mkdir(parents=True, exist_ok=True)
    save_loc = f"data/db/{filename}"
    response = requests.get(url)

    with open(save_loc, 'wb') as f:
        f.write(response.content)

    with zipfile.ZipFile(save_loc, 'r') as zip_ref:
        zip_ref.extractall("data/db/")

## Schéma de la base de données

<img src="https://cdn.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg" />

## Se connecter à la base de données

In [10]:
conn = sqlite3.connect('data/db/chinook.db')
c = conn.cursor()

## Trouver l'identifiant de Miles Davis

In [11]:
artist = 'Miles Davis'

query1 = "SELECT ArtistId from artists WHERE Name = ?"
c.execute(query1, (artist,))
artist_id = c.fetchone()

print(artist_id)

(68,)


## Trouver les identifiants d'albums de Miles Davis

In [12]:
query2 = "SELECT * from albums where ArtistId = ?"
albums_ids = []
for row in c.execute(query2, artist_id):
    albums_ids.append(str(row[0]))

print(albums_ids)

['48', '49', '157']


## Trouver les morceaux des albums de Miles Davis

In [9]:
query3 = f'SELECT Name, AlbumId FROM tracks WHERE AlbumId IN ({",".join(albums_ids)})'
songs = set()
for row in c.execute(query3):
    songs.add(row[0])

print(f"\n{len(songs)} distinct songs found:\n")
for song in sorted(songs):
    print(song)


36 distinct songs found:

'Round Midnight
Black Satin
Blues For Pablo
Blues For Pablo (Alternate Take)
Bye Bye Blackbird
Compulsion
E.S.P.
Generique
I Don't Wanna Be Kissed (By Anyone But You)
I Don't Wanna Be Kissed (By Anyone But You) (Alternate Take)
Jean Pierre (Live)
Jeru
Lament
Little Church (Live)
Miles Ahead
Miles Runs The Voodoo Down
My Funny Valentine (Live)
My Ship
Nefertiti
New Rhumba
Now's The Time
Petits Machins (Little Stuff)
Portia
So What
Someday My Prince Will Come
Springsville
Springsville (Alternate Take)
Summertime
Tempus Fugit
The Duke
The Maids Of Cadiz
The Meaning Of The Blues
The Meaning Of The Blues/Lament (Alternate Take)
The Pan Piper
Time After Time
Walkin'
